# Autoregressive vs. Masked Diffusion Transformers for Translation (En - Fr)

### 1. Introduction
Autoregressive transformer models have become the dominant paradigm for text generation, achieving state-of-the-art performance across tasks like translation and summarization. These models, such as GPT-2 and GPT-3, generate text sequentially by predicting one token at a time from left to right, conditioning each prediction on all previous tokens.

While this approach produces highly coherent text, it suffers from an inherent sequential bottleneck that prevents parallelization and results in slow inference times. Recently, diffusion models have emerged as a promising alternative, generating all tokens in parallel through an iterative denoising process.

In this notebook, we implement and compare these two paradigms using identical 44M-parameter architectures trained on the OPUS Books English-to-French dataset.

#### 1.1 Environment set-up

In [ ]:
!pip install transformers torch tqdm numpy pandas scikit-learn
!pip install sacrebleu rouge-score bert-score
!pip install datasets tokenizers
!pip install hf_transfer

In [ ]:
import os
import sys
import math
import time
import random
import datetime
import json
import sacrebleu
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors
from datasets import load_dataset
from tqdm.auto import tqdm
import collections

# Standardizing Seeds for Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Configuration and Data


#### 2.1 Dataset Description
We use the OPUS Books dataset for English-to-French machine translation, comprising 127,000 sentence pairs. The English sentences average 15.2 tokens in length, while French translations average 16.8 tokens.


#### 2.2 Tokenization Strategy
We developed a custom Byte Pair Encoding (BPE) tokenizer with a vocabulary size of 20,000 tokens, trained jointly on both English and French text. This balances expressiveness with computational efficiency.

In [ ]:
class Config:
    # Data & Tokenizer
    tokenizer_path = "custom_opus_bpe.json"
    max_input_len = 128
    max_output_len = 128

    # Model Architecture
    d_model = 512
    n_heads = 8
    n_layers = 8
    d_ff = 2048
    dropout = 0.1
    activation = "silu"
    vocab_size = 20000

    pad_token_id = None
    sep_token_id = None
    mask_token_id = None
    eos_token_id = None

    # Training
    batch_size = 16
    accumulate_grad_batches = 4
    learning_rate = 1e-4
    weight_decay = 0.01
    warmup_steps = 1000
    epochs = 50
    clip_grad = 1.0

    # Diffusion Specifics
    timesteps = 100
    sampling_steps = 50

    # Logging
    log_dir = "logs"
    sample_dir = "samples"
    checkpoint_dir = "checkpoints"
    save_top_k = 3

    def to_dict(self):
        return {k: v for k, v in Config.__dict__.items() if not k.startswith('__') and not callable(v)}

config = Config()

# Create directories
os.makedirs(config.log_dir, exist_ok=True)
os.makedirs(config.sample_dir, exist_ok=True)
os.makedirs(config.checkpoint_dir, exist_ok=True)

In [ ]:
# 1. Load the OPUS Books dataset (English-French subset)
print("Loading OPUS Books (en-fr)...")
dataset = load_dataset("opus_books", "en-fr", split="train")

# 2. Format it for your specific pipeline
data = []

print("Formatting dataset...")
for item in dataset:
    question = item['translation']['en']
    answer = item['translation']['fr']
    data.append({'question': question, 'answer': answer})

# 3. Save to CSV
data_path = 'french_dataset.csv'
df_fr = pd.DataFrame(data)
df_fr.to_csv(data_path, index=False)
print(f"Saved {data_path} with {len(df_fr)} pairs.")

# 4. Train Custom BPE Tokenizer
print("Training Custom BPE Tokenizer...")

# Initialize BPE tokenizer
tokenizer_obj = Tokenizer(models.BPE())
tokenizer_obj.pre_tokenizer = pre_tokenizers.Whitespace()

# Define special tokens
# We include <sep> and <mask> specifically for our architecture
special_tokens = ["<pad>", "<unk>", "<s>", "</s>", "<sep>", "<mask>"]

trainer = trainers.BpeTrainer(
    vocab_size=20000,
    special_tokens=special_tokens,
    min_frequency=2
)

# Train on the questions and answers
# We stream from the dataframe to avoid massive memory usage if data is large
def batch_iterator():
    for i in range(0, len(df_fr), 1000):
        yield df_fr[i : i + 1000]["question"].astype(str).tolist()
        yield df_fr[i : i + 1000]["answer"].astype(str).tolist()

tokenizer_obj.train_from_iterator(batch_iterator(), trainer=trainer)

# Save the tokenizer JSON
tokenizer_obj.save(config.tokenizer_path)
print(f"Tokenizer saved to {config.tokenizer_path} with vocab size {tokenizer_obj.get_vocab_size()}")

#### 2.3 Dataset Class and Loader


In [ ]:
class PreTokenizedDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_input, max_output):
        self.examples = []
        df = pd.read_csv(data_path, on_bad_lines='skip', engine='python').dropna(subset=['question', 'answer'])

        # Batch tokenize using tokenizer.batch_encode_plus (Much faster than loops)
        print("Pre-tokenizing dataset... this takes a moment but speeds up training.")

        questions = df['question'].astype(str).tolist()
        answers = df['answer'].astype(str).tolist()

        q_enc = tokenizer(questions, truncation=True, max_length=max_input, add_special_tokens=False)
        a_enc = tokenizer(answers, truncation=True, max_length=max_output, add_special_tokens=False)

        sep_id = tokenizer.convert_tokens_to_ids('<sep>')
        eos_id = tokenizer.eos_token_id

        for q, a in zip(q_enc['input_ids'], a_enc['input_ids']):
            # Construct: Q + SEP + A + EOS
            full_ids = q + [sep_id] + a + [eos_id]
            # Truncate
            if len(full_ids) > (max_input + max_output + 1):
                full_ids = full_ids[:(max_input + max_output + 1)]
            self.examples.append(torch.tensor(full_ids, dtype=torch.long))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

In [ ]:
# --- Tokenizer Setup (Custom BPE) ---
# We wrap the custom JSON in a Transformers-compatible class
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=config.tokenizer_path,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)

# Add sep token manually as it's not a standard HF attribute
tokenizer.add_special_tokens({'additional_special_tokens': ['<sep>']})

# Update Config with IDs
config.vocab_size = len(tokenizer)
config.pad_token_id = tokenizer.pad_token_id
config.sep_token_id = tokenizer.convert_tokens_to_ids('<sep>')
config.mask_token_id = tokenizer.convert_tokens_to_ids('<mask>')
config.eos_token_id = tokenizer.eos_token_id

print(f"Vocab size: {config.vocab_size}")
print(f"SEP token ID: {config.sep_token_id}")
print(f"MASK token ID: {config.mask_token_id}")
print(f"PAD token ID: {config.pad_token_id}")

def collate_fn(batch):
    # Pad sequences
    padded_ids = pad_sequence(batch, batch_first=True, padding_value=config.pad_token_id)
    # Create attention mask (1 for real tokens, 0 for pad)
    attention_mask = (padded_ids != config.pad_token_id).long()
    return padded_ids, attention_mask

# Instantiate Data Loaders using the new CSV
data_path = 'french_dataset.csv'

# Load the full dataset
# We use the PreTokenizedDataset class defined in the previous cell
full_dataset = PreTokenizedDataset(
    data_path, tokenizer, config.max_input_len, config.max_output_len
)

# Split Train/Val
val_ratio = 0.05
n_val = int(len(full_dataset) * val_ratio)
n_train = len(full_dataset) - n_val

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [n_train, n_val]
)

train_loader = DataLoader(
    train_dataset, batch_size=config.batch_size, shuffle=True,
    collate_fn=collate_fn, num_workers=2, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=config.batch_size, shuffle=False,
    collate_fn=collate_fn, num_workers=2, pin_memory=True
)

print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")

### 3. Methods: Model Architecture
We implemented two 8-layer transformer models with identical architectures to ensure fair comparison. Both models utilize 43,992,576 parameters and learned positional embeddings.

#### 3.1 Shared Components

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        var = torch.mean(x ** 2, dim=-1, keepdim=True)
        x_normed = x * torch.rsqrt(var + self.eps)
        return self.weight * x_normed

class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return F.silu(gate) * x

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.norm1 = RMSNorm(d_model)
        self.norm2 = RMSNorm(d_model)

        self.ffn = nn.Sequential(
            nn.Linear(d_model, 2 * d_ff),
            SwiGLU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None, key_padding_mask=None):
        residual = x
        x_norm = self.norm1(x)

        attn_out, _ = self.self_attn(
            x_norm, x_norm, x_norm,
            attn_mask=attn_mask,
            key_padding_mask=key_padding_mask,
            need_weights=False
        )
        x = residual + self.dropout(attn_out)

        # FFN
        residual = x
        x_norm = self.norm2(x)
        x = residual + self.ffn(x_norm)

        return x

class PositionalEmbedding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        seq_len = x.size(1)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0)
        return self.pe(positions)

#### 3.2 Autoregressive Model (ARM)
The autoregressive baseline uses standard causal masking to prevent attending to future tokens.

In [ ]:
class ARM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.token_emb = nn.Embedding(config.vocab_size, config.d_model)
        self.pos_emb = PositionalEmbedding(config.max_input_len + config.max_output_len + 10, config.d_model)
        self.dropout = nn.Dropout(config.dropout)

        self.layers = nn.ModuleList([
            TransformerDecoderLayer(config.d_model, config.n_heads, config.d_ff, config.dropout)
            for _ in range(config.n_layers)
        ])

        self.norm_final = RMSNorm(config.d_model)
        self.output_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

        self.output_head.weight = self.token_emb.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input_ids, attention_mask=None):

        seq_len = input_ids.size(1)

        x = self.token_emb(input_ids) + self.pos_emb(input_ids)
        x = self.dropout(x)

        causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=input_ids.device) * float('-inf'), diagonal=1)

        if attention_mask is not None:
            key_padding_mask = (attention_mask == 0)
        else:
            key_padding_mask = None

        # Transformer Layers
        for layer in self.layers:
            x = layer(x, attn_mask=causal_mask, key_padding_mask=key_padding_mask)

        x = self.norm_final(x)
        logits = self.output_head(x)
        return logits

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

#### 3.3 Masked Diffusion Model (MDM)
Our diffusion model follows the masked discrete diffusion framework inspired by LLaDA. During training, the model learns to predict original tokens at masked positions using bidirectional attention.

In [ ]:
class DiffusionLLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.token_emb = nn.Embedding(config.vocab_size, config.d_model)
        self.pos_emb = PositionalEmbedding(config.max_input_len + config.max_output_len + 10, config.d_model)

        self.dropout = nn.Dropout(config.dropout)

        # Re-use the exact same block structure
        self.layers = nn.ModuleList([
            TransformerDecoderLayer(config.d_model, config.n_heads, config.d_ff, config.dropout)
            for _ in range(config.n_layers)
        ])

        self.norm_final = RMSNorm(config.d_model)
        self.output_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

        # Weight tying
        self.output_head.weight = self.token_emb.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input_ids, attention_mask=None):

        x = self.token_emb(input_ids) + self.pos_emb(input_ids)
        x = self.dropout(x)

        if attention_mask is not None:
            key_padding_mask = (attention_mask == 0)
        else:
            key_padding_mask = None

        for layer in self.layers:
            x = layer(x, attn_mask=None, key_padding_mask=key_padding_mask)

        x = self.norm_final(x)
        logits = self.output_head(x)
        return logits

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

print("Re-initializing models with updated class structure...")
model_arm = ARM(config).to(device)
model_diff = DiffusionLLM(config).to(device)

model_arm = torch.compile(model_arm)
model_diff = torch.compile(model_diff)

params_arm = model_arm.count_parameters()
params_diff = model_diff.count_parameters()

print(f"ARM Parameters: {params_arm:,}")
print(f"Diffusion Parameters: {params_diff:,}")
diff_perc = abs(params_arm - params_diff) / params_arm * 100
print(f"Difference: {diff_perc:.4f}%")

### 4. Training Mechanism

#### 4.1 Masking and Loss Strategy
1) ARM: Minimizes cross-entropy loss for next-token prediction across all positions.
2) Diffusion: Randomly masks tokens based on a linear schedule. Cross-entropy loss is computed only on masked tokens.

In [ ]:
def get_mask_ratio(t, T, schedule="cosine"):
    if schedule == "linear":
        return t.float() / T
    elif schedule == "cosine":
        return torch.cos((t.float() / T) * math.pi * 0.5)
    return t.float() / T

def train_step_arm(model, batch):
    input_ids, attention_mask = batch
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    inputs = input_ids[:, :-1]
    targets = input_ids[:, 1:]
    att_mask = attention_mask[:, :-1]

    logits = model(inputs, attention_mask=att_mask)

    # Flatten
    loss = F.cross_entropy(logits.reshape(-1, config.vocab_size), targets.reshape(-1), ignore_index=config.pad_token_id)
    return loss

def train_step_diffusion(model, batch):
    input_ids, attention_mask = batch
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    batch_size, seq_len = input_ids.shape

    t = torch.rand(batch_size, device=device)

    ratio = torch.cos((t * math.pi * 0.5))

    # 2. Determine which tokens can be masked
    indices = torch.arange(seq_len, device=device).expand(batch_size, seq_len)

    # Find position of SEP token (argmax returns the first index of max value)
    sep_locs = (input_ids == config.sep_token_id).long().argmax(dim=1).unsqueeze(1)

    # Mask is true if index > sep_position (Only mask the Answer part)
    can_be_masked = indices > sep_locs

    # 3. Create Random Mask Probabilities
    probs = torch.rand(batch_size, seq_len, device=device)
    mask_mask = (probs < ratio.unsqueeze(1)) & can_be_masked & (input_ids != config.pad_token_id)

    # 4. Apply Mask
    masked_input = input_ids.clone()
    masked_input[mask_mask] = config.mask_token_id

    # 5. Forward
    logits = model(masked_input, attention_mask=attention_mask)

    # 6. Loss Calculation
    # We explicitly IGNORE the pad token so the model doesn't learn to predict it
    loss = F.cross_entropy(
        logits.view(-1, config.vocab_size),
        input_ids.view(-1),
        reduction='none',
        ignore_index=config.pad_token_id
    )

    loss = loss.view(batch_size, seq_len)

    # Only count loss on masked tokens
    masked_loss = loss * mask_mask.float()

    # 7. Stable Loss Aggregation
    # Instead of dividing by t (which causes exploding gradients/collapse),
    # we normalize by the actual count of masked tokens.
    num_masked_tokens = mask_mask.sum() + 1e-6

    return masked_loss.sum() / num_masked_tokens

#### 4.2 Training Loop
Both models are trained for 50 epochs using the AdamW optimizer with a learning rate of 1e-4.

In [ ]:
def train_model(model, model_type, train_loader, val_loader, config):
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs * len(train_loader))

    metrics_log = []

    print(f"Starting training for {model_type}...")

    for epoch in range(config.epochs):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs}")

        scaler = torch.amp.GradScaler('cuda')

        for step, batch in enumerate(pbar):
            optimizer.zero_grad()

            with torch.amp.autocast('cuda'): # Casts operations to FP16 automatically
                if model_type == "ARM":
                    loss = train_step_arm(model, batch)
                else:
                    loss = train_step_diffusion(model, batch)

                loss = loss / config.accumulate_grad_batches

            # Scale the loss
            scaler.scale(loss).backward()

            if (step + 1) % config.accumulate_grad_batches == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip_grad)

                scaler.step(optimizer)
                scaler.update()
                scheduler.step()

            total_loss += loss.item() * config.accumulate_grad_batches
            pbar.set_postfix({'loss': total_loss / (step + 1)})

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        val_loss = evaluate_loss(model, model_type, val_loader)
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # Save Checkpoint
        ckpt_path = os.path.join(config.checkpoint_dir, f"{model_type}_epoch_{epoch+1}.pt")
        torch.save(model.state_dict(), ckpt_path)

        # Generate Samples & Compute Text Metrics
        metrics = evaluate_generation(model, model_type, val_loader, config, epoch)
        metrics['epoch'] = epoch + 1
        metrics['train_loss'] = avg_train_loss
        metrics['val_loss'] = val_loss
        metrics_log.append(metrics)

        # Save logs
        pd.DataFrame(metrics_log).to_csv(os.path.join(config.log_dir, f"{model_type}_metrics.csv"), index=False)

def evaluate_loss(model, model_type, loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            if model_type == "ARM":
                loss = train_step_arm(model, batch)
            else:
                loss = train_step_diffusion(model, batch)
            total_loss += loss.item()
    return total_loss / len(loader)

### 5. Inference and Evaluation

#### 5.1 Generation Strategy
1) ARM: Generates tokens left-to-right (O(N) cost).
2) Diffusion: Uses semi-autoregressive block-wise sampling. High-confidence predictions are retained, while low-confidence tokens are re-masked for refinement.

In [ ]:
def generate_arm(model, prompt_ids, max_new_tokens=50, eos_token_id=None):
    """
    Standard Autoregressive Generation (Greedy).
    """
    model.eval()
    curr_ids = prompt_ids.clone()

    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(curr_ids)
            # Predict next token from the last position
            next_token_logits = outputs[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

            curr_ids = torch.cat([curr_ids, next_token], dim=1)

            # optional: stop if EOS is generated
            if eos_token_id is not None and (next_token == eos_token_id).all():
                break

    return curr_ids

def generate_diffusion(model, prompt_ids, target_len=50, steps=20, block_size=None, repetition_penalty=1.5, eos_token_id=None):
    """
    LLaDA-style Generation with:
    1. Greedy Decoding (Argmax) for stability
    2. Repetition Penalty to reduce stuttering
    3. EOS Truncation (stops generating if </s> is found)
    """
    model.eval()

    # Start with the prompt
    curr_ids = prompt_ids.clone()
    batch_size = curr_ids.shape[0]

    # Default to full parallel generation if block_size is not set
    if block_size is None or block_size <= 0:
        block_size = target_len

    # Determine how many blocks we need
    num_blocks = math.ceil(target_len / block_size)

    for block_idx in range(num_blocks):
        # 1. Determine length of this specific block
        current_block_len = min(block_size, target_len - (block_idx * block_size))

        # 2. Append MASK tokens for the current block ONLY
        context_len = curr_ids.shape[1]
        mask_token_id = config.mask_token_id

        mask_append = torch.full((batch_size, current_block_len), mask_token_id, device=curr_ids.device)
        curr_ids = torch.cat([curr_ids, mask_append], dim=1)

        # 3. Iterative Denoising
        for step in range(steps):
            t = 1.0 - ((step + 1) / steps) # Represents time going from 1 to 0
            ratio_masked = math.cos(t * math.pi * 0.5) # Apply cosine map

            with torch.no_grad():
                logits = model(curr_ids)

            # Focus only on the current block's logits
            block_logits = logits[:, context_len:, :]

            # --- Repetition Penalty ---
            if repetition_penalty > 1.0:
                for i in range(batch_size):
                    existing_tokens = curr_ids[i].unique()
                    for t_id in existing_tokens:
                        if t_id not in [config.pad_token_id, config.sep_token_id, config.mask_token_id]:
                            if block_logits[i, :, t_id].max() > 0:
                                block_logits[i, :, t_id] /= repetition_penalty
                            else:
                                block_logits[i, :, t_id] *= repetition_penalty

            probs = F.softmax(block_logits, dim=-1)

            # --- Greedy Decoding (Argmax) ---
            pred_ids = torch.argmax(probs, dim=-1)

            # Confidence calculation
            confidence = torch.max(probs, dim=-1).values

            # Re-masking logic
            num_to_mask = int(current_block_len * ratio_masked)

            if num_to_mask == 0:
                curr_ids[:, context_len:] = pred_ids
                break

            _, mask_indices = torch.topk(confidence, k=num_to_mask, dim=1, largest=False)
            curr_ids[:, context_len:] = pred_ids

            batch_indices = torch.arange(batch_size, device=curr_ids.device).unsqueeze(1).expand_as(mask_indices)
            current_block_ids = curr_ids[:, context_len:].clone()
            current_block_ids[batch_indices, mask_indices] = mask_token_id
            curr_ids[:, context_len:] = current_block_ids

    # --- EOS Truncation Logic ---
    # If eos_token_id is provided, we cut the tensor short
    if eos_token_id is not None:
        # Check the first sequence in the batch (assuming batch_size=1 for inference)
        # Find first occurrence of EOS in the generated part
        generated_part = curr_ids[0, prompt_ids.shape[1]:]
        eos_indices = (generated_part == eos_token_id).nonzero(as_tuple=True)[0]

        if len(eos_indices) > 0:
            first_eos = eos_indices[0].item()
            # Truncate: Keep prompt + generated up to EOS
            curr_ids = curr_ids[:, :prompt_ids.shape[1] + first_eos]

    return curr_ids

def clean_stutter(text):
    """
    Simple post-processing to remove immediate word repetitions.
    """
    words = text.split()
    if not words: return ""
    clean_words = [words[0]]
    for w in words[1:]:
        if w.lower() != clean_words[-1].lower():
            clean_words.append(w)
    return " ".join(clean_words)

#### 5.2 Evaluation Metrics
We evaluate using BLEU, ROUGE, and BERTScore.

In [ ]:
import sacrebleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score

def calculate_metrics(refs, preds):
    # BLEU
    bleu = sacrebleu.corpus_bleu(preds, [[r] for r in refs])

    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = np.mean([scorer.score(r, p)['rougeL'].fmeasure for r, p in zip(refs, preds)])

    # BERTScore (using distilbert for speed)
    try:
        P, R, F1 = bert_score(preds, refs, lang="en", verbose=False, model_type="distilbert-base-uncased")
        f1_score = F1.mean().item()
        p_score = P.mean().item()
        r_score = R.mean().item()
    except Exception as e:
        print(f"BERTScore failed: {e}")
        f1_score, p_score, r_score = 0.0, 0.0, 0.0

    return {
        "bleu": bleu.score,
        "rouge_l": rouge_l,
        "bert_p": p_score,
        "bert_r": r_score,
        "bert_f1": f1_score
    }

def evaluate_generation(model, model_type, loader, config, epoch):
    model.eval()
    inputs_list = []
    refs_list = []
    preds_list = []

    # Process a few batches only for speed
    limit_batches = 5

    print(f"Generating samples for {model_type}...")
    with torch.no_grad():
        for i, (input_ids, _) in enumerate(loader):
            if i >= limit_batches: break
            input_ids = input_ids.to(device)

            # Iterate over the batch
            for idx in range(input_ids.shape[0]):
                row = input_ids[idx]
                sep_idx = (row == config.sep_token_id).nonzero()
                if len(sep_idx) == 0: continue
                sep_pos = sep_idx.item()

                # 1. 2D Tensor for the Model (Batch Size = 1)
                prompt_ids_model = row[:sep_pos+1].unsqueeze(0)

                # 2. 1D Tensor for the Tokenizer (List of IDs)
                prompt_ids_decode = row[:sep_pos+1]

                target_ids = row[sep_pos+1:]

                # Decode using the 1D tensors
                ref_text = tokenizer.decode(target_ids, skip_special_tokens=False)
                prompt_text = tokenizer.decode(prompt_ids_decode, skip_special_tokens=False)

                # Generate
                if model_type == "ARM":
                    # Use the 2D model input
                    gen_ids = generate_arm(model, prompt_ids_model)
                    new_tokens = gen_ids[0, prompt_ids_model.shape[1]:]
                else:
                    tgt_len = target_ids.shape[0]
                    # Use the 2D model input
                    gen_ids = generate_diffusion(model, prompt_ids_model, target_len=tgt_len)

                    new_tokens = gen_ids[0, prompt_ids_model.shape[1]:]

                pred_text = tokenizer.decode(new_tokens, skip_special_tokens=False)

                inputs_list.append(prompt_text)
                refs_list.append(ref_text)
                preds_list.append(pred_text)

    # Save Samples
    df_samples = pd.DataFrame({
        'epoch': epoch + 1,
        'input': inputs_list,
        'reference': refs_list,
        'prediction': preds_list
    })

    # Create directory if it doesn't exist just in case
    os.makedirs(config.sample_dir, exist_ok=True)

    sample_file = os.path.join(config.sample_dir, f"{model_type}_epoch_{epoch+1}.csv")
    df_samples.head(5).to_csv(sample_file, index=False)
    print(f"Saved samples to {sample_file}")

    # Calculate Metrics
    if len(preds_list) > 0:
        try:
            metrics = calculate_metrics(refs_list, preds_list)
            print(f"Epoch {epoch+1} Metrics: {metrics}")
            return metrics
        except Exception as e:
            print(f"Skipping metrics calculation due to error (likely OOM): {e}")
            return {}
    return {}

### Execution and Results


#### 6.1 Execution

In [ ]:
# Train both models
# train_model(model_arm, "ARM", train_loader, val_loader, config)
# train_model(model_diff, "Diffusion", train_loader, val_loader, config)

#### 6.2 Model Loading & Qualitative Comparison


In [ ]:
import torch
import os
import sacrebleu
import numpy as np
from rouge_score import rouge_scorer
from bert_score import score as bert_score

In [ ]:
# --- 1. Configuration & Checkpoint Loading ---
# Paths to your saved checkpoints
CHECKPOINT_ARM = "checkpoints/ARM_epoch_40.pt"       # Adjust epoch as needed
CHECKPOINT_DIFF = "checkpoints/Diffusion_epoch_40.pt" # Adjust epoch as needed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Loading models on {device}...")

# Initialize Architectures (Must match training config)
try:
    model_arm = ARM(config).to(device)
    model_diff = DiffusionLLM(config).to(device)
except NameError:
    raise NameError("Model classes (ARM, Diffusion) not defined. Please run the Architecture cell first.")

# Load ARM State
if os.path.exists(CHECKPOINT_ARM):
    state_dict_arm = torch.load(CHECKPOINT_ARM, map_location=device)
    model_arm.load_state_dict(state_dict_arm)
    model_arm.eval()
    print(f"✅ ARM loaded from {CHECKPOINT_ARM}")
else:
    print(f"❌ ARM checkpoint not found at {CHECKPOINT_ARM}")

# Load Diffusion State
if os.path.exists(CHECKPOINT_DIFF):
    state_dict_diff = torch.load(CHECKPOINT_DIFF, map_location=device)
    model_diff.load_state_dict(state_dict_diff)
    model_diff.eval()
    print(f"✅ Diffusion loaded from {CHECKPOINT_DIFF}")
else:
    print(f"❌ Diffusion checkpoint not found at {CHECKPOINT_DIFF}")

In [ ]:
# --- 2. Unified Translation Wrapper ---

def translate_unified(text, model, model_type, penalty=1.5):
    """
    Unified inference wrapper for both ARM and Diffusion models.
    """
    # Common Tokenization
    input_ids = tokenizer.encode(text)
    
    if model_type == "ARM":
        # ARM Input: Just the prompt IDs
        input_tensor = torch.tensor([input_ids], device=device)
        
        # Standard Greedy Generation
        gen_ids = generate_arm(
            model, 
            input_tensor, 
            max_new_tokens=50, 
            eos_token_id=config.eos_token_id
        )
        
        # Slice: Remove the prompt to get only the translation
        new_tokens = gen_ids[0, len(input_ids):]
        
        # Decode
        raw_output = tokenizer.decode(new_tokens, skip_special_tokens=True)
        return raw_output.strip()

    elif model_type == "Diffusion":
        # Diffusion Input: Prompt + SEP token
        input_tensor = torch.tensor([input_ids + [config.sep_token_id]], device=device)
        
        # Dynamic Target Length: French is ~1.3x English length
        tgt_len = int(len(input_ids) * 1.3) + 2
        
        # LLaDA-style Generation
        gen_ids = generate_diffusion(
            model, 
            input_tensor, 
            target_len=tgt_len, 
            steps=50, 
            repetition_penalty=penalty,
            eos_token_id=config.eos_token_id 
        )
        
        # Slice: Find SEP and take everything after
        try:
            sep_loc = (gen_ids[0] == config.sep_token_id).nonzero(as_tuple=True)[0].item()
            new_tokens = gen_ids[0, sep_loc+1:]
            
            # Manual EOS truncation check
            if config.eos_token_id in new_tokens:
                end_loc = (new_tokens == config.eos_token_id).nonzero(as_tuple=True)[0][0].item()
                new_tokens = new_tokens[:end_loc]
                
            raw_output = tokenizer.decode(new_tokens, skip_special_tokens=True)
            
            # Uses clean_stutter from the previous cell
            return clean_stutter(raw_output).strip()
            
        except Exception as e:
            return f"[Error in generation: {e}]"
    
    return "[Unknown Model Type]"


In [ ]:
import sacrebleu
import numpy as np
import pandas as pd
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# ==========================================
# 1. SETUP: Ladder of Difficulty Dataset
# ==========================================
test_data = [
    # --- Level 1: Basics ---
    ("The sun is shining.", "Le soleil brille."),
    ("He opened the door.", "Il a ouvert la porte."),
    ("She looked at him.", "Elle le regarda."),
    ("I am happy.", "Je suis heureux."),
    ("The dog runs fast.", "Le chien court vite."),
    ("The cat sleeps on the bed.", "Le chat dort sur le lit."),
    ("We eat apples.", "Nous mangeons des pommes."),
    ("They drink water.", "Ils boivent de l'eau."),
    ("My father is tall.", "Mon père est grand."),
    ("The car is red.", "La voiture est rouge."),

    # --- Level 2: Simple Grammar ---
    ("I do not know the answer.", "Je ne connais pas la réponse."),
    ("Where is the house?", "Où est la maison ?"),
    ("The little boy played in the garden.", "Le petit garçon jouait dans le jardin."),
    ("She is a very beautiful woman.", "C'est une très belle femme."),
    ("He said nothing to me.", "Il ne m'a rien dit."),
    ("Why are you sad?", "Pourquoi es-tu triste ?"),
    ("I did not see the movie.", "Je n'ai pas vu le film."),
    ("Do you have a pen?", "As-tu un stylo ?"),
    ("The big house is old.", "La grande maison est vieille."),
    ("We are not ready yet.", "Nous ne sommes pas encore prêts."),

    # --- Level 3: Compound Sentences ---
    ("I went to the market and I bought some bread.", "Je suis allé au marché et j'ai acheté du pain."),
    ("He wanted to come, but he was too tired.", "Il voulait venir, mais il était trop fatigué."),
    ("If it rains, we will stay inside.", "S'il pleut, nous resterons à l'intérieur."),
    ("She told me that she loved him.", "Elle m'a dit qu'elle l'aimait."),
    ("When I arrived, the door was already open.", "Quand je suis arrivé, la porte était déjà ouverte."),
    ("I called him but he did not answer.", "Je l'ai appelé mais il n'a pas répondu."),
    ("She was reading while I was cooking.", "Elle lisait pendant que je cuisinais."),
    ("You can stay or you can go.", "Tu peux rester ou tu peux partir."),
    ("Because it was late, we went home.", "Comme il était tard, nous sommes rentrés à la maison."),
    ("He thinks that we are lost.", "Il pense que nous sommes perdus."),

    # --- Level 4: Complex Grammar ---
    ("The man who is standing there is my brother.", "L'homme qui se tient là est mon frère."),
    ("It is necessary that you leave immediately.", "Il est nécessaire que vous partiez immédiatement."),
    ("Despite the rain, they continued their journey.", "Malgré la pluie, ils ont continué leur voyage."),
    ("She wondered if he would ever return to this place.", "Elle se demandait s'il reviendrait un jour à cet endroit."),
    ("He spoke with a voice that was both calm and terrifying.", "Il parlait d'une voix à la fois calme et terrifiante."),
    ("The book that I read yesterday was boring.", "Le livre que j'ai lu hier était ennuyeux."),
    ("I doubt that he is telling the truth.", "Je doute qu'il dise la vérité."),
    ("She asked me where I had bought this coat.", "Elle m'a demandé où j'avais acheté ce manteau."),
    ("Before you leave, please close the window.", "Avant que tu partes, s'il te plaît ferme la fenêtre."),
    ("The woman whose car was stolen is crying.", "La femme dont la voiture a été volée pleure."),

    # --- Level 5: Literary/Abstract ---
    ("A deep silence reigned in the room, broken only by the ticking of the clock.", "Un grand silence régnait dans la pièce, troublé seulement par le tic-tac de l'horloge."),
    ("He felt a strange sensation, as if someone were watching him from the shadows.", "Il éprouvait une étrange sensation, comme si quelqu'un l'observait depuis l'ombre."),
    ("It was the best of times, it was the worst of times.", "C'était le meilleur des temps, c'était le pire des temps."),
    ("Suddenly, a loud cry rang out through the night, freezing his blood.", "Soudain, un grand cri retentit dans la nuit, lui glaçant le sang."),
    ("She sat by the window, watching the dead leaves fall slowly to the ground.", "Elle était assise près de la fenêtre, regardant les feuilles mortes tomber lentement vers le sol."),
    ("The wind howled through the trees like a wounded animal.", "Le vent hurlait à travers les arbres comme un animal blessé."),
    ("In the distance, the mountains rose sharply against the pale sky.", "Au loin, les montagnes se dressaient brusquement contre le ciel pâle."),
    ("He knew in his heart that this was the end of the journey.", "Il savait dans son cœur que c'était la fin du voyage."),
    ("The ancient castle stood silent, guarding its secrets for centuries.", "L'ancien château se tenait silencieux, gardant ses secrets depuis des siècles."),
    ("A heavy fog covered the city, hiding the streets from view.", "Un brouillard épais couvrait la ville, dérobant les rues à la vue."),
]

# ==========================================
# 2. EVALUATION LOOP (Unified)
# ==========================================
references = []
predictions_arm = []
predictions_diff = []

print(f"{'='*20} RUNNING INFERENCE ON {len(test_data)} SENTENCES {'='*20}")

for i, (source, ref) in enumerate(test_data):
    # 1. ARM Generation
    pred_arm = translate_unified(source, model_arm, "ARM")
    if not pred_arm.strip(): pred_arm = "."

    # 2. Diffusion Generation (with penalty)
    pred_diff = translate_unified(source, model_diff, "Diffusion", penalty=1.5)
    if not pred_diff.strip(): pred_diff = "."

    predictions_arm.append(pred_arm)
    predictions_diff.append(pred_diff)
    references.append(ref)

    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1} sentences...")

# ==========================================
# 3. HELPER: Metric Calculator
# ==========================================
def get_metrics(preds, refs, model_name):
    print(f"\nCalculating metrics for {model_name}...")

    # BLEU
    bleu = sacrebleu.corpus_bleu(preds, [[r] for r in refs])

    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge1, rougeL = [], []
    for r, p in zip(refs, preds):
        scores = scorer.score(r, p)
        rouge1.append(scores['rouge1'].fmeasure)
        rougeL.append(scores['rougeL'].fmeasure)

    # BERTScore
    try:
        P, R, F1 = bert_score(preds, refs, lang="fr", verbose=False)
        bert_f1 = F1.mean().item()
    except Exception as e:
        print(f"BERTScore warning: {e}")
        bert_f1 = 0.0

    return {
        "BLEU": bleu.score,
        "ROUGE-1": np.mean(rouge1),
        "ROUGE-L": np.mean(rougeL),
        "BERTScore": bert_f1
    }

# ==========================================
# 4. COMPUTE METRICS
# ==========================================
metrics_arm = get_metrics(predictions_arm, references, "ARM")
metrics_diff = get_metrics(predictions_diff, references, "Diffusion")

# ==========================================
# 5. FINAL REPORT & COMPARISON
# ==========================================

# A. Metrics Table
results_df = pd.DataFrame([metrics_diff, metrics_arm], index=["Diffusion", "ARM"])
print("\n" + "="*40)
print("       FINAL PERFORMANCE COMPARISON")
print("="*40)
print(results_df.round(4))
print("="*40 + "\n")

# B. Detailed List Display
print(f"{'='*20} DETAILED GENERATION LOG {'='*20}")
for i, (src, ref) in enumerate(test_data):
    print(f"[{i+1}] Input:      {src}")
    print(f"    Reference:  {ref}")
    print(f"    ARM:        {predictions_arm[i]}")
    print(f"    Diffusion:  {predictions_diff[i]}")
    print("-" * 50)

# 6. Conclusions and Future Work

## **6.1 Summary of Findings**
This work presented a systematic comparison between Autoregressive Models (ARM) and Masked Diffusion Models (MDM) for English-to-French machine translation using a 44M parameter scale.

Our experiments demonstrate that **autoregressive models substantially outperform diffusion models** in this specific setting:
* **Quality:** The ARM achieved a **BLEU score of 30.42**, which is approximately **2.2× higher** than the diffusion model's score of 13.77.
* **Coherence:** Qualitative analysis shows the ARM produces fluent, grammatically correct translations, whereas the diffusion model struggles with long-range dependencies and occasionally introduces repetition artifacts.
* **Efficiency:** The autoregressive approach converges significantly faster, reaching lower loss values within the 50-epoch constraint.

## **6.2 Limitations and Analysis**
While the autoregressive model is superior at this scale, these results likely reflect **training efficiency differences** rather than fundamental algorithmic limitations of diffusion.
* **Scale:** Diffusion models often require larger parameter scales (e.g., 7B+) for emergent capabilities like global planning to manifest.
* **Training Time:** Diffusion models typically require 5–10× more training steps to reach parity with ARMs, which our 50-epoch limit did not permit.
* **Architecture:** Our diffusion implementation lacked advanced features like Rotary Positional Embeddings (RoPE) or Grouped-Query Attention (GQA), which are standard in state-of-the-art models like LLaDA.

## **6.3 Future Work**
To fully realize the potential of diffusion models for text generation, future research should focus on:
1.  **Scaling Experiments:** Training at 1B+ parameters to unlock emergent bidirectional reasoning capabilities.
2.  **Extended Training Schedules:** Increasing the training budget to allow the diffusion model to fully converge.
3.  **Advanced Architectures:** Incorporating modern techniques such as RoPE and GQA to better exploit bidirectional context.
4.  **Task Diversity:** Evaluating performance on tasks that inherently benefit from non-sequential generation, such as text infilling, constrained generation, or iterative revision.